# Part 3 

In [ ]:
import requests
import time
import pandas as pd
import threading
from concurrent.futures import ThreadPoolExecutor

def rate_limited_get(url, rate_limiter):
    """Handles API rate limiting."""
    while True:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 429:
            time.sleep(1)  # Wait and retry if rate-limited
        else:
            response.raise_for_status()

def get_author_id(name):
    """Retrieve author ID from OpenAlex given the author's name."""
    url = f"https://api.openalex.org/authors?search={name}&per_page=1"
    data = rate_limited_get(url, rate_limiter)
    return data['results'][0]['id'] if data['results'] else None

def get_works_by_author(author_id):
    """Retrieve works by author ID and apply filters."""
    works = []
    page_url = f"https://api.openalex.org/works?filter=author.id:{author_id},cited_by_count:>10,authorships_count:<10"
    while page_url:
        data = rate_limited_get(page_url, rate_limiter)
        works.extend(data['results'])
        page_url = data.get('meta', {}).get('next_cursor')
    return works

def fetch_data_for_authors(author_names):
    """Fetch works data for multiple authors in parallel."""
    author_ids = [get_author_id(name) for name in author_names]
    author_ids = [aid for aid in author_ids if aid]
    all_works = []
    with ThreadPoolExecutor(max_workers=5) as executor:
        results = executor.map(get_works_by_author, author_ids)
        for res in results:
            all_works.extend(res)
    return all_works

def process_works(works):
    """Process works into DataFrame format."""
    papers_data = []
    abstracts_data = []
    for work in works:
        papers_data.append({
            'id': work['id'],
            'publication_year': work.get('publication_year', 'Unknown'),
            'cited_by_count': work.get('cited_by_count', 0),
            'author_ids': [a['author']['id'] for a in work.get('authorships', [])]
        })
        abstracts_data.append({
            'id': work['id'],
            'title': work.get('title', 'Unknown'),
            'abstract_inverted_index': work.get('abstract_inverted_index', '')
        })
    return pd.DataFrame(papers_data), pd.DataFrame(abstracts_data)
